In [1]:
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

### Function to extract pixel IDs from a given page

### Extraxts pixel ids from all live snapshots downloaded

In [24]:
# Function to extract pixel IDs from a given HTML snapshot
def process_snapshot(snapshot_path):
    pixel_ids = set()

    # Read the HTML content from the file
    with open(snapshot_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Pattern 1: Extract IDs from script src pattern
    script_pattern = r'<script src="https://connect\.facebook\.net/signals/config/(\d+)'
    ids_from_script = re.findall(script_pattern, content)
    pixel_ids.update(ids_from_script)

    # Pattern 2: Extract IDs from fbq("init", ...) pattern
    fbq_pattern = r'fbq\("init","(\d+)"\);'
    ids_from_fbq = re.findall(fbq_pattern, content)
    pixel_ids.update(ids_from_fbq)

    # Return the list of unique IDs
    return list(pixel_ids)

# Function to process all HTML files in a folder and generate the CSV
def generate_pixel_ids_csv(folder_path, output_csv):
    # List to store the data
    data = []

    # Get all HTML files from the folder
    html_files = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.html')]

    for file_name in tqdm(html_files, desc="Processing HTML files"):
        file_path = os.path.join(folder_path, file_name)
        
        # Get the website name from the HTML file name (without the '.html' extension)
        website_name = os.path.splitext(file_name)[0]
        
        # Process the HTML file to extract pixel IDs
        pixel_ids = process_snapshot(file_path)
        
        # Add the website name and the pixel IDs to the data list
        data.append([website_name, pixel_ids])

    df = pd.DataFrame(data, columns=['Website', 'Pixel IDs'])
    df.to_csv(output_csv, index=False)

folder_path = 'live_websites' #change as per your reqs
output_csv = 'pixelHistoryLive.csv' #don't change

# Run the function to generate the CSV
generate_pixel_ids_csv(folder_path, output_csv)


Processing HTML files: 100%|██████████| 24/24 [00:00<00:00, 495.73it/s]


In [25]:
pd.read_csv('pixelHistoryLive.csv')

,Website,Pixel IDs
0,riteaid.com,['1264059003707256']
1,gstatic.com,[]
2,googlesyndication.com,[]
3,linkedin.com,[]
4,amazon.com,[]
5,bing.com,[]
6,twitter.com,[]
7,icloud.com,[]
8,googletagmanager.com,[]
9,wordpress.org,[]


In [26]:
import pandas as pd

csv_with_multiple_columns = 'pixelHistory.csv'  # Your CSV file with multiple columns
pixel_live_csv = 'pixelHistoryLive.csv'  # The CSV we generated with pixel IDs

df1 = pd.read_csv(csv_with_multiple_columns)
df2 = pd.read_csv(pixel_live_csv)

# Merge the two DataFrames on the 'website' column, keeping all rows from df1
# If there's no match in df2, it will place NaN (which we'll replace with None later)
df_merged = pd.merge(df1, df2[['Website', 'Pixel IDs']], left_on='website', right_on='Website', how='left')

# Drop the duplicate 'Website' column since we already have 'website'
df_merged = df_merged.drop(columns=['Website'])

# Rename 'Pixel IDs' column to 'live' for the output
df_merged = df_merged.rename(columns={'Pixel IDs': 'live'})

# Replace NaN values in 'live' column with None
df_merged['live'] = df_merged['live'].apply(lambda x: None if pd.isna(x) else x)

output_csv = 'pixelHistoryComplete.csv'
df_merged.to_csv(output_csv, index=False)

print(f"New CSV with 'live' column saved as {output_csv}")

New CSV with 'live' column saved as pixelHistoryComplete.csv


In [27]:
pd.read_csv('pixelHistoryComplete.csv')["live"].value_counts()

live
[]    18
Name: count, dtype: int64